## Project: Development of a reduced pediatric injury prediction model
Created by: Thomas Hartka, MD, MS  
Date created: 12/20/20  
  
This notebook evaluates the reduced variable-set model found by Bayesian model averaging and variable number analysis.  This uses ten-fold cross-validation, but uses different fold assignments than the original analysis.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import KFold
import scipy.stats as st
from itertools import combinations

## Set outcome

In [2]:
# outcome of interest
#  ISS -> ISS>=16
#  TIL -> any injury on target injury list
outcome = "ISS"

## Read in data

In [3]:
peds = pd.read_csv("../Data/Peds-2010_2018.csv")

In [4]:
# filter out CISS cases if using TIL because TIL is only valid for AIS98
if outcome == "TIL":
    peds = peds[peds.dataset=='NASS']

## Set variables

In [5]:
predictors = ['sex','age_5_9', 'age_10_14','age_15_18',
              'prop_restraint','no_restraint','front_row', 
              'dvtotal','pdof_rear','pdof_nearside','pdof_farside', 
              'rolled','multicoll','ejection']

if outcome == "ISS":
    predictors_reduced = ['dvtotal','ejection','no_restraint', 'pdof_nearside']
    response_train = 'iss16'
    
elif outcome == "TIL":
    predictors_reduced = ['dvtotal','ejection','no_restraint', 'pdof_nearside', 'multicoll']
    response_train = 'target_inj'
    
else:
    raise Exception("Outcome not valid") 
    
response_test = 'iss16'

## Create new fold assignments

In [6]:
# drop existing fold columns
peds = peds.drop(columns=['fold10x','fold5x'])

In [7]:
num_folds = 10

# initial fold column
folds = pd.Series(-1).repeat(len(peds)).reset_index(drop=True)

# set up k-fold generator
kf = KFold(n_splits=num_folds, shuffle=True, random_state=2716057)

# get splits for data
kf.get_n_splits(peds)
        
# interate through folds and assign fold number to cases
for i,row_list in enumerate(kf.split(peds)):
      folds.loc[row_list[1]] = i
    
peds['fold'] = folds

## Cross validation logistic regression function

In [8]:
def log_reg_cv(data, predictors, response_train, response_test, threshold=0.5):
    '''
    This function performs lass regression using 10-fold cross validation.  
    It returns a dataframe with the coefficients for each fold and auc.
    
    Parameters:
        data - data to analyze
        predictors - list of columns for predictors
    Returns:
        cofficient/AUC - DataFrame(contains AUC, fold, cofficients for model)
    '''
    
    # get folds
    folds = np.sort(data['fold'].unique())
    
    # create dataframe for results
    results = pd.DataFrame(columns=['num_vars','fold']+predictors+['AUC']+ \
                           ['tp','fp','tn','fn','tpwgt','fpwgt','tnwgt','fnwgt'] + \
                           ['sens','spec','senswgt','specwgt'] + \
                           ['total','totalwgt'])
    
    # set up LR model
    lr_mod = LogisticRegression(random_state=1819, penalty='none',solver='saga',max_iter=10000)
    
    # loop through folds
    for fold in folds:
        # separate fold train/test data
        train = data[data['fold']!=fold].copy()
        test = data[data['fold']==fold].copy()

        # fit regression model
        lr_fit = lr_mod.fit(train[predictors], train[response_train])

        # predict on fold test data
        pred = lr_fit.predict_proba(test[predictors])
        test['pred'] = pred[:,1]
        
        # calc AUC
        fpr, tpr, thresholds = metrics.roc_curve(test[response_test], pred[:,1], pos_label=1)
        AUC = metrics.auc(fpr, tpr)

        # gather results
        fold_results = [len(predictors), fold]
        for i,var in enumerate(predictors):
            fold_results.append(lr_fit.coef_[0,i])
        fold_results += [AUC]
        
        # get performance characteristics
        tp = test[(test.pred>=threshold) & (test[response_test]==1)].casewgt.count()
        fp = test[(test.pred>=threshold) & (test[response_test]==0)].casewgt.count()
        tn = test[(test.pred<=threshold) & (test[response_test]==0)].casewgt.count()
        fn = test[(test.pred<=threshold) & (test[response_test]==1)].casewgt.count()
        
        # get weighted performance characteristics
        tpwgt = test[(test.pred>=threshold) & (test[response_test]==1)].casewgt.sum()
        fpwgt = test[(test.pred>=threshold) & (test[response_test]==0)].casewgt.sum()
        tnwgt = test[(test.pred<=threshold) & (test[response_test]==0)].casewgt.sum()
        fnwgt = test[(test.pred<=threshold) & (test[response_test]==1)].casewgt.sum()
        
        # calc sensitivity and specificity 
        sens = tp / (tp+fn)
        spec = tn / (tn+fp)
        senswgt = tpwgt / (tpwgt+fnwgt)
        specwgt = tnwgt / (tnwgt+fpwgt)
        
        # calc totals
        total = tp+fp+tn+fn
        totalwgt = tpwgt+fpwgt+tnwgt+fnwgt
        
        fold_results += [tp]+[fp]+[tn]+[fn]+\
                        [tpwgt]+[fpwgt]+[tnwgt]+[fnwgt]+\
                        [sens]+[spec]+[senswgt]+[specwgt]+\
                        [total]+[totalwgt]

        # store AUC
        fold_series = pd.Series(fold_results, index = results.columns)
        results = results.append(fold_series, ignore_index=True)
    
    
    return results

## Compare AUCs of baseline and reduced models

In [9]:
%%time
results = log_reg_cv(peds,predictors,response_train, response_test)
results_reduced = log_reg_cv(peds,predictors_reduced,response_train, response_test)

CPU times: user 1min 42s, sys: 12.5 s, total: 1min 54s
Wall time: 1min 36s


In [10]:
def results_CI(results,sig_dig=2):
    mean = results.AUC.mean()
    sd = results.AUC.std()

    # calculate lower and upper 95% CI
    ll = round(mean - 1.96*sd, sig_dig)
    ul = round(mean + 1.96*sd, sig_dig)
    
    mean = round(mean, sig_dig)
    
    print(mean,"[",ll,"-",ul,"]")

In [11]:
results_CI(results,3)

0.906 [ 0.882 - 0.93 ]


In [12]:
results_CI(results_reduced,3)

0.897 [ 0.866 - 0.929 ]


## Examine probability thresholds

In [13]:
%%time
thresholds = pd.DataFrame(columns=['threshold','sens','spec','senswgt','specwgt'])
                                   
for i in np.linspace(0.0,0.1,11):
    thres_res = log_reg_cv(peds,predictors_reduced,response_train, response_test,threshold=i)
    res = [i]+[thres_res.sens.mean()]+[thres_res.spec.mean()]+[thres_res.senswgt.mean()]+[thres_res.specwgt.mean()]
    
    thres_means = pd.Series(res, index = thresholds.columns)
    thresholds = thresholds.append(thres_means, ignore_index=True)
    
    print(i)

0.0
0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
CPU times: user 6min 42s, sys: 71.4 ms, total: 6min 42s
Wall time: 6min 42s


In [14]:
thresholds

,threshold,sens,spec,senswgt,specwgt
0,0.00,1.000000,0.000000,1.000000,0.000000
1,0.01,0.983289,0.364319,0.904283,0.493870
2,0.02,0.940899,0.634568,0.867519,0.768554
3,0.03,0.890135,0.746707,0.783900,0.872912
4,0.04,0.851492,0.806002,0.737446,0.909171
5,0.05,0.805023,0.843584,0.705335,0.932053
6,0.06,0.769325,0.871112,0.657229,0.948131
7,0.07,0.735570,0.891048,0.613084,0.958425
8,0.08,0.700372,0.905042,0.591187,0.962664
9,0.09,0.669171,0.914551,0.556788,0.967999


## Calculate under and over triage rates

In [22]:
triage = thresholds[['threshold','senswgt','specwgt']].copy()

# calculate undertriage and overtriage
triage['Undertriage rate'] = 1-triage.senswgt 
triage['Overtriage rate'] = 1-triage.specwgt 

triage[['threshold','Undertriage rate','Overtriage rate']].rename(columns={'threshold':'Probability Threshold'})

,Probability Threshold,Undertriage rate,Overtriage rate
0,0.00,0.000000,1.000000
1,0.01,0.095717,0.506130
2,0.02,0.132481,0.231446
3,0.03,0.216100,0.127088
4,0.04,0.262554,0.090829
5,0.05,0.294665,0.067947
6,0.06,0.342771,0.051869
7,0.07,0.386916,0.041575
8,0.08,0.408813,0.037336
9,0.09,0.443212,0.032001
